# Batchnorm
<font color=red>Normalization用在affine层后面，activation前面。</font>

## 1. 计算

### **Forward**
$norm = \frac {(x - mean)}{\sqrt{var + eps}}$\
$out=norm*gamma+beta$

### **Backward** 
$dnorm = gamma * dout$ \
$dx = \frac{\partial L}{\partial x} + \frac{\partial L}{\partial mean}*\frac{d mean}{d x} + \frac{\partial L}{\partial var}*\frac{d var}{d x}=part1 + part2+part3$ \
1. <font color=blue>**计算part1：**</font>\
$part1=\frac{\partial L}{\partial x} = \frac{1}{\sqrt{var + eps}} * dnorm$
2. <font color=blue>**计算part2：**</font>\
$\frac{\partial L}{\partial mean}= -np.sum(\frac{1}{\sqrt{var + eps}} *dnorm, axis=0)$\
$\frac{d mean_{j}}{d x_{ij}}=\frac{1}{N}$\
$part2=\frac{\partial L}{\partial mean}*\frac{d mean}{d x}$
$=-\frac{1}{N}*\frac{1}{\sqrt{var + eps}} * dnorm$ \
错！$mean_j$影响norm所有行，不止第$i$行 \
$part2=-\frac{1}{N}*\frac{1}{\sqrt{var + eps}} * np.sum(dnorm,axis=0)*np.ones(x.shape)$

3. <font color=blue>**计算part3：**</font>

$$\begin{align}
\frac{\partial L}{\partial var}
&=np.sum(-\frac{1}{2}*\frac{x-mean}{(var+eps)^{\frac{3}{2}}}*dnorm, axis=0)\\
\frac{\partial var_i}{\partial x_i}
&=\frac{2}{N}*(x_i-mean_i),\ \ \  \frac{\partial var_i}{\partial mean_i}=0\\
\frac{d var_i}{d x_i}
&=\frac{\partial var_i}{\partial mean_i}*\frac{d mean_i}{d x_i}+\frac{\partial var_i}{\partial x_i}=\frac{2}{N}*(x_i-mean_i)\\
part3
& = \frac{\partial L}{\partial var}*\frac{d var}{d x}\\
& = \frac{2}{N}*(x-mean)*np.sum(-\frac{1}{2}*\frac{x-mean}{(var+eps)^{\frac{3}{2}}}*dnorm, axis  = 0)\\
&= -\frac{1}{N}*(x-mean)*np.sum(\frac{1}{var+eps}*norm*dnorm, axis  = 0)\\
& = -\frac{1}{N}*\frac{x-mean}{var+eps}*np.sum(norm*dnorm, axis  = 0)\\
part3
&=-\frac{1}{N}*\frac{1}{\sqrt{var+eps}}*norm*np.sum(norm*dnorm, axis=0)
\end{align}
$$


## 2. 效果分析：以MLP为例

1. **不同batchsize下，用和不使用bn的accuracy对比图**
 - **epoch=50，baseline是no batchnorm，batch=5**
<img src="pics/batchnorm1.png" alt="alt text" width="560"/>
 - **epoch=50，baseline是no batchnorm，batch=10(图上标记有误)**
<img src="pics/batchnorm2.png" alt="alt text" width="560"/>
 - **epoch=50，baseline是no batchnorm，batch=50(图上标记有误)**
<img src="pics/batchnorm3.png" alt="alt text" width="560"/>


2. **batchsize对batchnorm效果的影响和原因分析**
    1. **batchsize较小的时候，batchnorm不仅不提升，反而降低training accuracy**。**batchsize越大，training accuracy越高**。随着batchsize打到一定数量后，做batchnorm的training accuracy也能达到不做batchnorm的模型水平。
    **分析：**其他条件相同的情况下，做batchnorm相当于增加了Regularization，使模型更不容易发生overfit，而且，batchsize越小，由于抽样带来的noise越大，Regularization strength也越大。所以当batchsize仅仅为5的时候，不用batchnorm会发生overfit，而用了batchnorm则不会。随着batchsize增加，Regularization strength下降，training accuracy就会增加。
    2. 做了batchnorm的模型中，batchsize越大，**validation accuracy和training accuracy提升的速度更快**。***[这是batchnorm的最大价值点：加速优化过程]***
    **分析：**batchsize越大，batch mean和batch variance值的波动更小，并且更能体现数据本身的真实的统计特征。This leads to more accurate normalization.从而使batchnorm的效果更好。
    3. 做了batchnorm的模型中，当训练达到一定epoch之后，**不同的batchsize对应的best validation accuracy区别不大**。
    **分析：**当迭代epoch数量足够多了之后，即使batchsize不同，但running average of mean and variance的值趋于一致，从test阶段的计算规则可知，这会使得validation的结果和training不同，后者的accuracy会趋于一致。

## 3. 使用batchnorm遇到的问题

1. 由于batchnorm的效果受batchsize的影响较大。导致复杂的网络中，如果batchsize受硬件限制而很小的话，batchnorm的效果就很有限。比如online learning，此时batchsize=1。
2. 无法用到RNN上。因为句子长度不同会导致在batch里面计算feature的均值时，其波动特别大。因为很多feature都是sparse的，在一个batch里面甚至都不会出现，feature mean的波动就会很大。此外，inference阶段如果有很长的句子，那么training阶段获得的全局均值和方差对它很可能不使用

为了解决这个问题，就有了layernorm和groupnorm等其他方法。